### **data organization and initial analysis**

In [171]:
import numpy as np
import pandas as pd
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# importing data set
df = pd.read_csv('Bitcoin Historical Data.csv')

# fixing dataset so that it can be interpreted by the model
df['Price'] = df['Price'].str.replace(',','')
df['Open'] = df['Open'].str.replace(',','')
df['High'] = df['High'].str.replace(',','')
df['Low'] = df['Low'].str.replace(',','')
df['Change %'] = df['Change %'].str.replace('%', '')

df['Price'] = df['Price'].astype(float)
df['Open'] = df['Open'].astype(float)
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)
df['Change %'] = df['Change %'].astype(float)

df['Vol.'] = df['Vol.'].str.replace(',','')
df['Vol.'] = df['Vol.'].str.replace('K','')
df['Vol.'] = df['Vol.'].str.replace('M','')
df['Vol.'] = df['Vol.'].str.replace('B','')
df['Vol.'] = df['Vol.'].astype(float)
df['Vol.'] = df['Vol.'] * 1000

df['Change %'] = df['Change %'] / 100

df['Fake_Index'] = np.array([i for i in range(0,len(df))])

# logic to determine whether a user should have bought or sold on this given day, originally derived later in code
buy_sell = []
for i in range(len(df)):
    if (df['Price'].iloc[i] > df['Open'].iloc[i]) and (abs(df['Price'].iloc[i] - df['Open'].iloc[i]) / df['Open'].iloc[i] > 0.016):
        buy_sell.append('buy')
    else:
        buy_sell.append('sell')

df['BuySell'] = np.array(buy_sell)

### **modeling**

In [179]:
X = df[['Fake_Index', 'Open', 'High', 'Low', 'Vol.', 'Change %']] # features (can be changed, r^2 doesn't change much though)
y = df['Price'] # target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.50)

model = LinearRegression() # linearity of dataset made it pretty easy to use a linear model to predict the prices just based on the opening and volume that day
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(r2_score(y_test, y_pred))

0.9993951587367179


In [180]:
y_pred = pd.Series(y_pred)
y_test = pd.Series(y_test)

train_data = pd.concat([X_train,y_train], axis=1)
train_data.drop(columns='Fake_Index', inplace=True)

orig_index = y_test.index.to_list()
y_test.reset_index(drop=True, inplace=True)

results = pd.concat([y_pred,y_test], axis=1, ignore_index=True)
results['Index'] = orig_index
results.set_index('Index', inplace=True)

results_new = pd.concat([results,X_test], axis=1)
results_new.rename(columns={0:'Predicted'}, inplace=True)
results_new.rename(columns={1:'Actual'}, inplace=True)

final = pd.merge(results_new, df['Date'], left_index=True, right_index=True, how='inner')
final.drop(columns='Fake_Index', inplace=True)

### **logic to determine whether to buy or sell**

In [181]:
buy_sell_pred = []
buy_sell_actual = []

for i in range(len(final)):
    if (final['Predicted'].iloc[i] > final['Open'].iloc[i]) and (abs(final['Predicted'].iloc[i] - final['Open'].iloc[i]) / final['Open'].iloc[i] > 0.016):
        buy_sell_pred.append('buy')
    else:
        buy_sell_pred.append('sell')
    
    if final['Actual'].iloc[i] > final['Open'].iloc[i] and (abs(final['Predicted'].iloc[i] - final['Open'].iloc[i]) / final['Open'].iloc[i] > 0.016):
        buy_sell_actual.append('buy')
    else:
        buy_sell_actual.append('sell')

final['BuySell_actual'] = np.array(buy_sell_actual)
final['BuySell_pred'] = np.array(buy_sell_pred)

final['BuySell_diff'] = final['BuySell_actual'] == final['BuySell_pred']

final = final[['Date', 'Predicted', 'Open', 'High', 'Low', 'Vol.', 'Change %', 'BuySell_pred']]
final.rename(columns={'Predicted':'Price'}, inplace=True)
final.rename(columns={'BuySell_pred':'BuySell'}, inplace=True)